### Notebook to parse scraped html to produce cleaned text of SCC decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/)

Dataset & Code to be cited as: 

    Sean Rehaag, "Supreme Court of Canada Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/scc/>.

Notes:

(1) Data Source: [Supreme Court of Canada](https://www.scc-csc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Supreme Court of Canada website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Supreme Court of Canada

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Supreme Court of Canada website's [Terms of Use](https://www.scc-csc.ca/terms-avis/notice-enonce-eng.aspx).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 



In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json
import random

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/SCC/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/scc_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/scc_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/scc_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')

# set years sought
start_year = 1877
end_year = 2023

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)

# Manually fix one error in the data in english version of 2008scc29
file_en = pathlib.Path('d:/scraping/DATA/DECISIONS/SCC/BULK/ARCHIVE/HTML/EN/2008/2008scc29=2022-09-01T04_14_02.645930.json')
if file.exists():
    with open(file_en) as f:
        data = json.load(f)
    
# get the df row with citation '2008 SCC 29', iterate through the columns, and replace the values with the values from the json file
for col in df.columns:
    df.loc[df['case_citation'] == '2008 SCC 29', col] = data[col]

#df


15604


100%|██████████| 15604/15604 [00:19<00:00, 800.38it/s]


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = ciation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

df['text'] = df.html.progress_apply(get_text)



100%|██████████| 15589/15589 [08:27<00:00, 30.75it/s]


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 15589/15589 [01:00<00:00, 257.35it/s]


In [7]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# reindex
df = df.reset_index(drop=True)

In [8]:
df.head()

,citation,citation2,year,name,language,decision_date,source_url,scraped_timestamp,unofficial_text
0,(1877) 1 SCR 110,,1877,Boak et al. v. The Merchant's Marine Insurance...,en,1877-01-23,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31,Boak et al. v. The Merchant's Marine Insurance...
1,(1877) 1 SCR 114,,1877,Smyth v. McDougall,en,1877-02-01,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31,Smyth v. McDougall\nCollection\nSupreme Court ...
2,(1877) 1 SCR 117,,1877,The Queen v. Laliberté,en,1877-02-03,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31,The Queen v. Laliberté\nCollection\nSupreme Co...
3,(1877) 1 SCR 145,,1877,Brassard et al. v. Langevin,en,1877-02-28,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31,Brassard et al. v. Langevin\nCollection\nSupre...
4,(1877) 1 SCR 235,,1877,Johnstone v. The Minister & Trustees of St. An...,en,1877-06-28,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31,Johnstone v. The Minister & Trustees of St. An...


In [9]:
df.tail()

,citation,citation2,year,name,language,decision_date,source_url,scraped_timestamp,unofficial_text
15584,2023 CSC 5,,2023,R. c. Metzger,fr,2023-03-03,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2023-04-13,R. c. Metzger\nCollection\nJugements de la Cou...
15585,2023 CSC 6,,2023,R. c. Downes,fr,2023-03-10,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2023-04-13,R. c. Downes\nCollection\nJugements de la Cour...
15586,2023 CSC 7,,2023,R. c. Chatillon,fr,2023-03-15,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2023-07-03,R. c. Chatillon\nCollection\nJugements de la C...
15587,2023 CSC 8,,2023,R. c. McColman,fr,2023-03-23,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2023-07-03,R. c. McColman\nCollection\nJugements de la Co...
15588,2023 CSC 9,,2023,R. c. Breault,fr,2023-04-13,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2023-07-03,R. c. Breault\nCollection\nJugements de la Cou...


### Export data

In [10]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [11]:
# export cleaned df to parquet
df.to_parquet(out_path_parquet)

In [12]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 147/147 [00:02<00:00, 72.21it/s]
